API für DinoProject

Methoden:

POST /game: Erstellt eine neue Session und gibt einen Redirect zurück
 
GET /highscore : Liefert alle Session-IDs aus der DB zurück <br/>
POST /highscore : Erstellt eine neue Session in der DB

GET /highscore/{id} : Liefert zu einer Session-ID den Highscore zurück <br/>
PUT /highscore/{id} : Aktualisiert bei einer Session den Highscore

In [1]:
from flask import Flask, request
from flask_restful import Resource, Api
from flask_cors import CORS
from sqlalchemy import create_engine
from json import dumps
from flask import jsonify, redirect
import optparse
import mysql.connector
import uuid

app = Flask(__name__)
cors = CORS(app, resources={r"*": {"origins": "*"}}) # CORS
api = Api(app)


# Datenbank konfigurieren und Verbindung herstellen
mydb = mysql.connector.connect(
    host = "webengineering.ins.hs-anhalt.de",
    port = 32192,
    user = "root",
    passwd = "ga4wm19",
    database = "dinoProject"
)

mycursor = mydb.cursor()
mydb.disconnect() # Vorläufig Verbindung schließen

In [2]:
# Klasse um eine neue Session zu erstellen
class createSession(Resource):
    def post(self):
        sessionID = str(uuid.uuid4().hex)
        return redirect('http://localhost:8080/game/' + sessionID, code=303)

# Klasse für Updaten und Abfragen eines spezifischen Highscores
class getOrAddHighscore(Resource):
    def get(self, id):
        sql = "SELECT * FROM dinoProject.highscoreTable WHERE sessionID = '" + id + "'"
        mydb.reconnect(attempts = 1 , delay = 0)
        mycursor.execute(sql)
        dbResult = mycursor.fetchall()
        
        if len(dbResult) == 1 :
            session = dbResult[0]
            return jsonify({'sessionID': session[0], 'highscore': session[1]})
        else:
            return jsonify({'status': 'none or more than one highscore found', 'statuscode': 409})
        
    def post(self, id):
        print(request.json)
        if not request.json or not "highscore" in request.json or not "sessionID" in request.json:
            return jsonify({'status': 'could not update highscore', 'statuscode': 409})
        else:
            sql = "INSERT INTO dinoProject.highscoreTable (sessionID, highscore) VALUES (%s, %s) ON DUPLICATE KEY UPDATE highscore = %s"
            values = (id, request.json["highscore"], request.json["highscore"])
            mydb.reconnect(attempts = 1 , delay = 0)
            mycursor.execute(sql, values)
            mydb.commit()
            mydb.disconnect()
            return jsonify({'status': 'updated highscore with id: ' + request.json["sessionID"], 'statuscode': 200})

In [ ]:
# Klassen an entsprechende URLs binden
api.add_resource(createSession, '/game')
api.add_resource(getOrAddHighscore, '/highscore/<id>')

# Run für Testzwecke, startet auf localhost:5000
if __name__ == "__main__":
    app.run(port=4000)

# Run für Dockercontainer, mit Portangabe
#if __name__ == "__main__":
 #   parser = optparse.OptionParser(usage="python server.py -p ")
  #  parser.add_option('-p', '--port', action='store', dest='port', help='The port to listen on.')
   # (args, _) = parser.parse_args()
    #if args.port == None:
     #   print "Missing required argument: -p/--port"
      #  sys.exit(1)
    #app.run(host='0.0.0.0', port=int(args.port), debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:4000/ (Press CTRL+C to quit)
127.0.0.1 - - [04/Sep/2019 15:01:45] "GET /highscore/6d02fd1e9d2143649d2b21d13a77d9e0 HTTP/1.1" 200 -
127.0.0.1 - - [04/Sep/2019 15:01:50] "POST /highscore/6d02fd1e9d2143649d2b21d13a77d9e0 HTTP/1.1" 200 -


None


127.0.0.1 - - [04/Sep/2019 15:03:49] "POST /game HTTP/1.1" 303 -
127.0.0.1 - - [04/Sep/2019 15:03:50] "GET /highscore/ee5f3f08de2b4ebf80a7cde917014a17 HTTP/1.1" 200 -
127.0.0.1 - - [04/Sep/2019 15:03:55] "OPTIONS /highscore/ee5f3f08de2b4ebf80a7cde917014a17 HTTP/1.1" 200 -


{'sessionID': 'ee5f3f08de2b4ebf80a7cde917014a17', 'highscore': 320}


...

127.0.0.1 - - [04/Sep/2019 15:22:59] "POST /highscore/2a613f78fd3847eba7c1a518fbff23e1 HTTP/1.1" 200 -
127.0.0.1 - - [04/Sep/2019 15:23:06] "OPTIONS /highscore/2a613f78fd3847eba7c1a518fbff23e1 HTTP/1.1" 200 -


{'sessionID': '2a613f78fd3847eba7c1a518fbff23e1', 'highscore': 312}


127.0.0.1 - - [04/Sep/2019 15:23:06] "POST /highscore/2a613f78fd3847eba7c1a518fbff23e1 HTTP/1.1" 200 -
127.0.0.1 - - [04/Sep/2019 15:23:22] "OPTIONS /highscore/2a613f78fd3847eba7c1a518fbff23e1 HTTP/1.1" 200 -


{'sessionID': '2a613f78fd3847eba7c1a518fbff23e1', 'highscore': 556}


127.0.0.1 - - [04/Sep/2019 15:23:23] "POST /highscore/2a613f78fd3847eba7c1a518fbff23e1 HTTP/1.1" 200 -
